<a href="https://colab.research.google.com/github/izmare/Flux_Dev_i2i/blob/main/FluxDevi2i.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#@title ## Setup and Model Download
#@markdown This cell will set up the environment and download the necessary models for image generation.

%cd /content
!git clone -b totoro4 https://github.com/camenduru/ComfyUI /content/TotoroUI
%cd /content/TotoroUI

!pip install -q torchsde einops diffusers accelerate xformers==0.0.27.post2 gradio==3.35.2
!apt -y install -qq aria2

#@markdown ### Choose Models to Download
download_flux_dev = True #@param {type:"boolean"}
download_flux_schnell = False #@param {type:"boolean"}

# Download FLUX.1-dev all-in-one model if selected
if download_flux_dev:
    !aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/camenduru/FLUX.1-dev/resolve/main/flux1-dev-fp8-all-in-one.safetensors -d /content/TotoroUI/models/checkpoints -o flux1-dev-fp8-all-in-one.safetensors

# Download FLUX.1-schnell all-in-one model if selected
if download_flux_schnell:
    !aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/camenduru/FLUX.1-schnell/resolve/main/flux1-schnell-fp16-all-in-one.safetensors -d /content/TotoroUI/models/checkpoints -o flux1-schnell-fp16-all-in-one.safetensors

!wget https://huggingface.co/camenduru/FLUX.1-dev/resolve/main/test.png -O /content/test.png

/content
Cloning into '/content/TotoroUI'...
remote: Enumerating objects: 13250, done.
remote: Total 13250 (delta 0), reused 0 (delta 0), pack-reused 13250 (from 1)
Receiving objects: 100% (13250/13250), 7.26 MiB | 8.80 MiB/s, done.
Resolving deltas: 100% (8951/8951), done.
/content/TotoroUI
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.4/50.4 kB 1.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 20.8/20.8 MB 42.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.5/12.5 MB 64.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 318.6/318.6 kB 18.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.2/61.2 kB 5.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.6/2.6 MB 31.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.4/76.4 kB 6.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 5.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 141.9/141.9 kB 11.5 MB/s eta 0:00

In [ ]:
#@title ## Image Generation with Gradio Interface
#@markdown Use this cell to launch a Gradio interface for generating images.

import os
import random
import torch
import numpy as np
from PIL import Image
import gradio as gr
import nodes
from nodes import NODE_CLASS_MAPPINGS
from totoro_extras import nodes_custom_sampler
from totoro_extras import nodes_flux
from totoro_extras import nodes_post_processing
from totoro import model_management

CheckpointLoaderSimple = NODE_CLASS_MAPPINGS["CheckpointLoaderSimple"]()
FluxGuidance = nodes_flux.NODE_CLASS_MAPPINGS["FluxGuidance"]()
RandomNoise = nodes_custom_sampler.NODE_CLASS_MAPPINGS["RandomNoise"]()
BasicGuider = nodes_custom_sampler.NODE_CLASS_MAPPINGS["BasicGuider"]()
KSamplerSelect = nodes_custom_sampler.NODE_CLASS_MAPPINGS["KSamplerSelect"]()
BasicScheduler = nodes_custom_sampler.NODE_CLASS_MAPPINGS["BasicScheduler"]()
SamplerCustomAdvanced = nodes_custom_sampler.NODE_CLASS_MAPPINGS["SamplerCustomAdvanced"]()
VAELoader = NODE_CLASS_MAPPINGS["VAELoader"]()
VAEDecode = NODE_CLASS_MAPPINGS["VAEDecode"]()
EmptyLatentImage = NODE_CLASS_MAPPINGS["EmptyLatentImage"]()
ImageScaleToTotalPixels = nodes_post_processing.NODE_CLASS_MAPPINGS["ImageScaleToTotalPixels"]()

def closestNumber(n, m):
    q = int(n / m)
    n1 = m * q
    if (n * m) > 0:
        n2 = m * (q + 1)
    else:
        n2 = m * (q - 1)
    if abs(n - n1) < abs(n - n2):
        return n1
    return n2

@torch.inference_mode()
def generate_image(positive_prompt, width, height, seed, steps, sampler_name, scheduler, guidance, model_choice, input_image_path):
    global unet, clip, vae
    unet, clip, vae = CheckpointLoaderSimple.load_checkpoint(model_choice)

    cond, pooled = clip.encode_from_tokens(clip.tokenize(positive_prompt), return_pooled=True)
    cond = [[cond, {"pooled_output": pooled}]]
    cond = FluxGuidance.append(cond, guidance)[0]
    noise = RandomNoise.get_noise(seed)[0]
    guider = BasicGuider.get_guider(unet, cond)[0]
    sampler = KSamplerSelect.get_sampler(sampler_name)[0]
    sigmas = BasicScheduler.get_sigmas(unet, scheduler, steps, 1.0)[0]

    if input_image_path:
        image = nodes.LoadImage().load_image(input_image_path)[0]
        latent_image = ImageScaleToTotalPixels.upscale(image, "lanczos", 1.0)[0]
        latent_image = VAEEncode.encode(vae, latent_image)[0]
    else:
        latent_image = EmptyLatentImage.generate(closestNumber(width, 16), closestNumber(height, 16))[0]

    sample, sample_denoised = SamplerCustomAdvanced.sample(noise, guider, sampler, sigmas, latent_image)
    model_management.soft_empty_cache()
    decoded = VAEDecode.decode(vae, sample)[0].detach()

    # Save the output image with an incremental filename
    counter = 0
    output_filename = f"/content/output_image_{counter:04d}.png"
    while os.path.exists(output_filename):
        counter += 1
        output_filename = f"/content/output_image_{counter:04d}.png"
    Image.fromarray(np.array(decoded * 255, dtype=np.uint8)[0]).save(output_filename)

    return Image.fromarray(np.array(decoded * 255, dtype=np.uint8)[0])

with gr.Blocks(analytics_enabled=False) as demo:
    with gr.Row():
        with gr.Column():
            positive_prompt = gr.Textbox(lines=3, interactive=True, value="cute anime girl with massive fluffy fennec ears and a big fluffy tail blonde messy long hair blue eyes wearing a maid outfit with a long black dress with a gold leaf pattern and a white apron eating a slice of an apple pie in the kitchen of an old dark victorian mansion with a bright window and very expensive stuff everywhere", label="Prompt")
            width = gr.Slider(minimum=256, maximum=2048, value=1024, step=64, label="Width")
            height = gr.Slider(minimum=256, maximum=2048, value=1024, step=64, label="Height")
            seed = gr.Slider(minimum=0, maximum=18446744073709551615, value=0, step=1, label="Seed (0=random)")
            steps = gr.Slider(minimum=4, maximum=50, value=20, step=1, label="Steps")
            guidance = gr.Slider(minimum=0, maximum=20, value=3.5, step=0.5, label="Guidance")
            model_choice = gr.Dropdown(["flux1-dev-fp8-all-in-one.safetensors", "flux1-schnell-fp16-all-in-one.safetensors"], label="Model Choice", value="flux1-dev-fp8-all-in-one.safetensors")
            sampler_name = gr.Dropdown(["euler", "heun", "heunpp2", "heunpp2", "dpm_2", "lms", "dpmpp_2m", "ipndm", "deis", "ddim", "uni_pc", "uni_pc_bh2"], label="Sampler Name", value="euler")
            scheduler = gr.Dropdown(["normal", "sgm_uniform", "simple", "ddim_uniform"], label="Scheduler", value="simple")
            input_image_path = gr.Image(label="Input Image", type="filepath")
            generate_button = gr.Button("Generate")
        with gr.Column():
            output_image = gr.Image(label="Generated image", interactive=False)

    generate_button.click(fn=generate_image, inputs=[positive_prompt, width, height, seed, steps, sampler_name, scheduler, guidance, model_choice, input_image_path], outputs=output_image)

demo.queue().launch(inline=False, share=True, debug=True)

ModuleNotFoundError: No module named 'nodes'